## Mixed Integer Linear Programing 
### E-Scooter Sharing Scheme
Conditional Service 

In [78]:
import pandas as pd
import numpy as np
import itertools
import os
import collections
import time
from pulp import *

In [79]:
minimise = True
reduced = True
# 8, 12, 15
demand_percent = 12
# 0, 1, 2
# 0 - low
# 1 - default
# 2 - high
costs = 1
budget = 1

In [80]:
if minimise:
    problem = LpProblem("E-Scooter Allocation Carbon Minimisation", LpMinimize)
else:
    problem = LpProblem("E-Scooter Allocation Profit Maximisation", LpMaximize)

C:\Users\zoria\miniconda3\envs\py37\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [81]:
if reduced:
    df_distance = pd.read_csv("Data/Preliminary analysis Data/Model Data - Smaller set Distance.csv")
else:
    df_distance = pd.read_csv("Data/Model Data - 56loc Distance.csv")

In [82]:
df_distance.fillna(0, inplace=True)
distance = df_distance.values[:,1:]
distance_dict = { (x,y): distance[x][y] for x in range(distance.shape[0]) for y in range(distance.shape[1])}

In [83]:
locations = list(df_distance['Location'])
location_idx = np.arange(0, len(locations))
loc_count = len(locations)

In [84]:
if reduced:
    files = os.listdir('Data/Preliminary analysis Data/Reduced set Demand/')
else:
    files = os.listdir('Data/Final set Demand/')

In [106]:
d = dict()
for f in files:
    hour = int(f.split(sep='.', maxsplit=1)[0])
    if reduced:
        df = pd.read_csv('Data/Preliminary analysis Data/Reduced set Demand/' + f)
    else:
        df = pd.read_csv('Data/Final set Demand/' + f)
    demand_arr = df.values[:,1:]
    demand_arr = demand_arr*(demand_percent/15)
    for i in range(loc_count):
        for j in range(loc_count):
            demand_arr[i][j] = np.round(demand_arr[i][j],0)
    d[hour] = demand_arr

AttributeError: 'numpy.ndarray' object has no attribute 'float'

In [9]:
demand = collections.OrderedDict(sorted(d.items())) 

In [10]:
demand_dict = dict()
for i,(_,v) in enumerate(demand.items()):
    demand_dict.update({(x,y,i): v[x][y] for x in range(loc_count) for y in range(loc_count)})

In [11]:
# Parameters
M = sys.maxsize
# if reduced:
#     Nmax = 10
# else:
#     Nmax = 40
Zmax = 100
Zmin = 1
# Carbon Cost values
Cc_scooter_km = 7
Cc_fixed_scooter = 55
Cc_fixed_dock = 30
Cc_fixed_station = 28
# Cost relocation
Cc_r = 120

scalar = 1.25

Qmin = 0
# Cost values
# cost of maintaining one scooter per kilometer driven
Cp_scooter_km = 0.41
Cp_fixed_scooter = 0.38
Cp_fixed_dock = 0.92
Cp_fixed_station = 2.3
# Cost relocation
Cp_r = 0.2
# Price rate for kilometer driven
P_km = 0.6
# Price for pickup
P_init = 1
if reduced:
    B = 500
else:
    B = 1200

In [12]:
# Penalty
if reduced:
    df_penalty = pd.read_csv("Data/Preliminary analysis Data/Penalty Carbon Costs Smaller set.csv")
else:
    df_penalty = pd.read_csv("Data/Penalty Carbon Costs 56loc.csv")
C_pen = df_penalty.values[:, 1]
C_pen = C_pen*scalar

In [13]:
# Sets
T = np.arange(0,len(files)+1)
# each location at a given time
X = list(itertools.product(location_idx, T))
X2 = [(i,it) for (i,it) in X if sum([demand_dict[(i,j,it)] for j in location_idx if i !=j and it != T[-1]]) > 0]
A1 = [(xi, xj) for xi in X for xj in X if xi[0] != xj[0] and xi[1]+1==xj[1]]
# Relocation
A2 = [(xi, xj) for xi in X for xj in X if xi[0]!=xj[0] and xi[1]==T[-1] and xj[1]==T[0]]

In [14]:
# Decision Variables
Yi = LpVariable.dicts("Station Presence", location_idx,0,cat=const.LpBinary)
Zi = LpVariable.dicts("Size", location_idx, 0, cat=const.LpInteger)
# Relocation
Rij = LpVariable.dicts("#Relocated_Scooters", A2, 0, cat=const.LpInteger)
R = LpVariable.dicts('Relocation needed', A2, 0, cat=const.LpBinary)
Vit = LpVariable.dicts("#Available_Scooters",X,0,cat=const.LpInteger)
Ditj = LpVariable.dicts("#Used_Scooters", A1, 0 ,cat=const.LpInteger)
Xit = LpVariable.dicts("Availability binary", X2, 0, cat=const.LpBinary)
# if minimise:
#     O = LpVariable("Profit Max Objective variable", 0, cat=const.LpContinuous)
# else:
#     O = LpVariable("Carbon Min Objective variable", 0, cat=const.LpContinuous)

In [15]:
# Objective function
if minimise:
    problem+=lpSum(Ditj[((i,ti),(j,tj))]*distance_dict[(i,j)]*Cc_scooter_km for ((i,ti),(j,tj)) in A1) + \
    lpSum((demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))])*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1) + \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cc_r for ((i,ti), (j,tj)) in A2) + \
    lpSum(Vit[(i,t)]*Cc_fixed_scooter for (i,t) in X if t==0) + \
    lpSum(Zi[i]*Cc_fixed_dock for i in location_idx) + \
    lpSum(Yi[i]*Cc_fixed_station for i in location_idx), "Objective function"

else:
    problem+=lpSum(Ditj[((i,ti),(j,tj))]*(distance_dict[(i,j)]*(P_km-Cp_scooter_km) + P_init) for ((i,ti),(j,tj)) in A1) - \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cp_r for ((i,ti), (j,tj)) in A2) - \
    lpSum(Vit[(i,t)]*Cp_fixed_scooter for (i,t) in X if t==0) - \
    lpSum(Zi[i]*Cp_fixed_dock for i in location_idx) - \
    lpSum(Yi[i]*Cp_fixed_station for i in location_idx), "Objective function"


In [16]:
for (i,ti) in X:
    if ti !=0:
        problem+= Vit[(i,ti-1)] - lpSum(Ditj[((i,ti-1),(j,ti))] for j in location_idx if i !=j) + lpSum(Ditj[((j, ti-1),(i, ti))] for j in location_idx if i !=j) == Vit[(i,ti)], f"Availability Balance {(i,ti)}"


In [17]:
for (i,ti) in X:
    if ti!=T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) >=0, f"Stocked Scooters for {(i,ti)}"

In [18]:
# Relocation constraint
for i in location_idx:
    problem+=Vit[(i,0)] == Vit[(i,T[-1])] + lpSum(Rij[((j,T[-1]), (i,0))] for j in location_idx if i!=j) - lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j), f"Relocation balance for location {i}"

In [19]:
for a2 in A2:
    problem+=Rij[a2]<=M*R[a2], f"Relocation needed for {a2}"

In [20]:
for i in location_idx:
    problem+=lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j) <= Vit[(i,T[-1])], f"Relocation availability for location {i}"

In [21]:
for (i,ti) in X:
    problem+= Zi[i] >=Vit[(i,ti)], f"Size constraint for {(i,ti)}"

In [22]:
for (i,ti) in X:
    problem+=Vit[(i,ti)] <= Zmax*Yi[i], f"Maximum size constraint {(i,ti)}"

In [23]:
for i in location_idx:
    problem+=Yi[i]*Zmin <=Zi[i], f"Minimum size constraint for {i}"

In [24]:
for ((i,ti),(j,tj)) in A1:
    problem+=Ditj[((i,ti),(j,tj))]<=demand_dict[(i,j,ti)], f"Maximum Demand for {((i,ti),(j,tj))}"

In [25]:
# problem+=lpSum(Yi[i] for i in location_idx) <= Nmax, f"Maximum number of docks"

In [26]:
problem+=lpSum(Vit[(i,0)]*Cp_fixed_scooter for i in location_idx) + lpSum(Zi[i]*Cp_fixed_dock for i in location_idx) + lpSum(Yi[i]*Cp_fixed_station for i in location_idx)<=B, "Maximum Budget"

In [27]:
if not minimise:
    problem+=lpSum(v for v in Ditj.values())/lpSum(v for v in demand_dict.values()) >= Qmin, "Minimum satisfied demand"

In [28]:
for (i,ti) in X2:
    problem+= Xit[(i,ti)] <= (Vit[(i,ti)] + lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j) - lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j))/lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j), f"Xit value constraint for {(i,ti)}"

In [29]:
for (i,ti) in X2:
    if ti != T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) <= M*Xit[(i,ti)]

In [30]:
for (i,ti) in X2:
    if ti != T[-1]:
        problem+= lpSum(Ditj[((i,ti), (j,ti+1))] for j in location_idx if i!=j) >= M*(Xit[(i,ti)]-1) + lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j)

In [31]:
if minimise:
    if reduced:
        logf = "stats_carbon_reduced.log"
        problem.writeLP('Models/carbon_reduced_model.lp')
    else:
        logf = "stats_carbon_final.log"
        problem.writeLP('Models/carbon_model.lp')
else:
    if reduced:
        logf = "stats_profit_reduced.log"
        problem.writeLP('Models/profit_reduced_model.lp')
    else:
        logf = "stats_profit_final.log"
        problem.writeLP('Models/profit_model.lp')


In [32]:
solver = apis.PULP_CBC_CMD(logPath=logf, options=['DivingVectorlength on', 'DivingSome on'], gapRel=0.01)
start = time.time()
solver.actualSolve(problem)
end = time.time()
print(f'Running time: {str(int(end-start))}s')
LpStatus[problem.status]

C:\Users\zoria\miniconda3\envs\py37\lib\site-packages\pulp\apis\coin_api.py:146: UserWarning: `logPath` argument replaces `msg=1`. The output will be redirected to the log file.
  warnings.warn('`logPath` argument replaces `msg=1`. The output will be redirected to the log file.')
Running time: 966s


'Optimal'

In [33]:
obj_val = sum([Ditj[((i,ti),(j,tj))].value()*distance_dict[(i,j)]*Cc_scooter_km for ((i,ti),(j,tj)) in A1]) + \
    sum([(demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))].value())*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1]) + \
    sum([R[((i,ti), (j,tj))].value()*distance_dict[(i,j)]*Cc_r for ((i,ti), (j,tj)) in A2]) + \
    sum([Vit[(i,t)].value()*Cc_fixed_scooter for (i,t) in X if t==0]) + \
    sum([Zi[i].value()*Cc_fixed_dock for i in location_idx]) + \
    sum([Yi[i].value()*Cc_fixed_station for i in location_idx])
obj_val

146544.54687275004

In [34]:
value(problem.objective)

146544.54687275033

In [35]:
# Extracting the results
#(idx, LpVariable)
size_list = []
for _,v in Zi.items():
    size_list.append(v.value())  

In [36]:
station_list = []
for _,v in Yi.items():
    station_list.append(v.value()) 

In [37]:
df_results = pd.DataFrame(list(zip(locations, station_list, size_list)), columns=["Location","Station", "Size"])
df_results

,Location,Station,Size
0,Balgreen and Roseburn,1.0,8.0
1,Blackhall,1.0,4.0
2,Broughton North and Powderhall,1.0,12.0
3,Broughton South,1.0,19.0
4,Bruntsfield,1.0,6.0
5,Comely Bank,1.0,10.0
6,Dalry and Fountainbridge,1.0,7.0
7,Drylaw,1.0,9.0
8,Gorgie East,1.0,28.0
9,Hillside and Calton Hill,1.0,8.0


In [38]:
if minimise:
    if reduced:
        df_results.to_csv("results-carbon-reduced.csv", index=False)
    else:
        df_results.to_csv("results-carbon.csv", index=False)
else:
    if reduced:
        df_results.to_csv("results-profit-reduced.csv", index=False)
    else:
        df_results.to_csv("results-profit.csv", index=False)

In [39]:
total_Vit = dict()
for k,v in Vit.items():
    if k[1] not in total_Vit:
        total_Vit[k[1]] = int(v.varValue)
    else:
        total_Vit[k[1]]+=int(v.varValue)

In [40]:
total_scooters = total_Vit[0]
print("#Scooters: " + str(total_Vit[0]))

#Scooters: 184


In [41]:
total_demand = 0
for _,v in demand_dict.items():
    total_demand+=v

satisfied_demand = 0
for _,v in Ditj.items():
    satisfied_demand+=v.varValue
satisfied_demand

print(f'Satisfied demand: {satisfied_demand/total_demand}')
print(f'Avg. Scooter rides per day: {satisfied_demand/total_scooters}')

Satisfied demand: 0.5041367898510756
Avg. Scooter rides per day: 4.967391304347826


In [42]:
trips = dict()
for ((i,ti),(j,tj)) in Ditj.keys():
    if ti+5 not in trips.keys():
        trips[ti+5] = int(Ditj[((i,ti),(j,tj))].varValue)
    else:
        trips[ti+5] +=int(Ditj[((i,ti),(j,tj))].varValue)
trips

{5: 27,
 6: 80,
 7: 128,
 8: 100,
 9: 32,
 10: 16,
 11: 3,
 12: 5,
 13: 4,
 14: 48,
 15: 107,
 16: 132,
 17: 119,
 18: 65,
 19: 27,
 20: 11,
 21: 3,
 22: 3,
 23: 4}

In [43]:
demand_t = dict()
for (i,j,t), v in demand_dict.items():
    if t+5 not in demand_t.keys():
        demand_t[t+5] = int(demand_dict[(i,j,t)])
    else:
        demand_t[t+5] +=int(demand_dict[(i,j,t)])
demand_t

{5: 74,
 6: 181,
 7: 274,
 8: 156,
 9: 54,
 10: 25,
 11: 8,
 12: 13,
 13: 13,
 14: 86,
 15: 193,
 16: 193,
 17: 274,
 18: 156,
 19: 54,
 20: 25,
 21: 8,
 22: 13,
 23: 13}

In [44]:
satisfied_demand_t = dict()
for t in demand_t.keys():
    satisfied_demand_t[t] = trips[t] / demand_t[t]
satisfied_demand_t

{5: 0.36486486486486486,
 6: 0.4419889502762431,
 7: 0.46715328467153283,
 8: 0.6410256410256411,
 9: 0.5925925925925926,
 10: 0.64,
 11: 0.375,
 12: 0.38461538461538464,
 13: 0.3076923076923077,
 14: 0.5581395348837209,
 15: 0.5544041450777202,
 16: 0.6839378238341969,
 17: 0.4343065693430657,
 18: 0.4166666666666667,
 19: 0.5,
 20: 0.44,
 21: 0.375,
 22: 0.23076923076923078,
 23: 0.3076923076923077}

In [45]:
# df_satisfied_demand = pd.DataFrame(satisfied_demand_t.items())
# df_satisfied_demand.to_csv("proportion of satisfied demand per timestep profit.csv", index=False)

In [46]:
Obj_matrix = np.array([[165817.27824972876, 591.0938],[178363.35, 644.104999781913]])
df_obj = pd.DataFrame(Obj_matrix, columns=["Carbon", "Profit"])
df_obj["Obj"] = ["Carbon", "Profit"]

cols = df_obj.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_obj = df_obj[cols]
# df_obj.to_csv("Objective_matrix.csv", index = False)

In [47]:
for k,v in Rij.items():
    if v.varValue>0:
        print(f"Rij[{k}] = {v.varValue}")

Rij[((0, 19), (8, 0))] = 5.0
Rij[((1, 19), (7, 0))] = 1.0
Rij[((4, 19), (13, 0))] = 3.0
Rij[((7, 19), (0, 0))] = 3.0
Rij[((8, 19), (19, 0))] = 2.0
Rij[((10, 19), (12, 0))] = 3.0
Rij[((11, 19), (16, 0))] = 1.0
Rij[((11, 19), (19, 0))] = 1.0
Rij[((14, 19), (12, 0))] = 2.0


In [48]:
# for t in T:
#     print(f"Vit[(19, {t})] = {Vit[(19,t)].varValue}")

# for t in T:
#     if t==T[0]:
#         print(f"Vit[(19, {t})] = {Vit[(19,t)].varValue}")
#     if t!=T[0]:
#         leaving = sum([Ditj[((19,t-1),(j,t))].varValue for j in location_idx if j!=19])
#         arriving = sum([Ditj[((j,t-1), (19,t))].varValue for j in location_idx if j!=19])

#         print(f"arriving - leaving = {arriving} - {leaving} = {arriving-leaving}")
#         print(f"Vit[(19, {t})] = {Vit[(19,t)].varValue}")

In [49]:
# table_19 = dict()
# for t in T:
#     table_19[t+5] = [Vit[(19,t)].varValue]
#     if t!=T[0]:
#         leaving = sum([Ditj[((19,t-1),(j,t))].varValue for j in location_idx if j!=19])
#         arriving = sum([Ditj[((j,t-1), (19,t))].varValue for j in location_idx if j!=19])
#         table_19[t+4].append(leaving)
#         table_19[t+4].append(arriving)

# d = dict()
# for t in T:
#     if t!=19:
#         d_val = Xit[(19,t)].varValue
#         d[t+5] = d_val

In [50]:
# df_19 = pd.DataFrame.from_dict(table_19,orient='index', columns=['Available scooters','Leaving', "Arriving"])
# df_19.to_csv("Daily operations at Tollcross.csv", index=True)

In [51]:
fixed_cost = sum([Vit[(i,0)].value()*Cp_fixed_scooter for i in location_idx]) + sum([Zi[i].value()*Cp_fixed_dock for i in location_idx]) + sum([Yi[i].value()*Cp_fixed_station for i in location_idx])

In [52]:
fixed_cost

456.32

In [54]:
loc_trips = dict()

for l in location_idx:
    loc = locations[l]
    df = pd.DataFrame(columns=['Time', 'Available', 'Leaving', 'Arriving'])
    for t in T:
        available = Vit[(l,t)].value()
        leaving = sum([Ditj[((l,t),(j,t+1))].value() for j in location_idx if j!=l and t!=T[-1]])
        arriving = sum([Ditj[((j,t),(l,t+1))].value() for j in location_idx if j!=l and t!=T[-1]])
        time = f"{int(t+5)}:00-{int(t+5)}:59"
        df.loc[-1] = [time, int(available), int(leaving), int(arriving)]
        df.index = df.index + 1
    relocate_from = sum([Rij[((l,T[-1]), (j,0))].value() for j in location_idx if j!=l])
    relocate_to = sum([Rij[((j,T[-1]), (l,0))].value() for j in location_idx if j!=l])
    df.loc[-1] = ['Relocate', None, int(relocate_from), int(relocate_to)]
    df.index = df.index + 1
    loc_trips[loc] = df

In [55]:
import report_generator as r
r.station_stats(loc_trips)

C:\Users\zoria\miniconda3\envs\py37\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [63]:
import csv
variable_values = dict()
for v in problem.variables():
    variable_values[v.name] = v.value()

variable_values_df = pd.DataFrame.from_dict(variable_values, orient='index')
df_csv = variable_values_df.to_csv(index=True)
with open('variable_values_carbon_reduced.csv', 'w') as f:
    f.write(df_csv)


In [62]:
variable_values_df

,0
"#Available_Scooters_(0,_0)",6.0
"#Available_Scooters_(0,_1)",6.0
"#Available_Scooters_(0,_10)",0.0
"#Available_Scooters_(0,_11)",2.0
"#Available_Scooters_(0,_12)",4.0
...,...
Station_Presence_5,1.0
Station_Presence_6,1.0
Station_Presence_7,1.0
Station_Presence_8,1.0
